In [1]:
import torch
from utils import load_module

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [ ]:
config = load_module("./configs/diffusion_hippocampus.py")

variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/Subjects/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = config.get_context(device, variables)

In [2]:
config = load_module("./configs/qsm_deep_grey_matter.py")

variables = dict(DATASET_PATH="X:/Datasets/DGM/segmentation_3T_ps18_v3/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = config.get_context(device, variables)

In [4]:
untransformed_subject =  context.dataset.subjects[0]
print("Original labels:")
print(untransformed_subject.dgm['label_names'])

subject = context.dataset[0]
print("\nTransformed labels:")
print(subject.dgm['label_names'])

inverse_subject = subject.apply_inverse_transform(warn=False)
print("\nInverse transformed labels:")
print(inverse_subject.dgm['label_names'])

Original labels:
{'left_ventricle': 1, 'right_ventricle': 2, 'left_caudate': 3, 'right_caudate': 4, 'left_putamen': 5, 'right_putamen': 6, 'left_thalamus': 7, 'right_thalamus': 8, 'left_globus_pallidus': 9, 'right_globus_pallidus': 10, 'internal_capsule': 17, 'left_red_nucleus': 19, 'right_red_nucleus': 20, 'left_substantia_nigra': 21, 'right_substantia_nigra': 22, 'left_dentate_nucleus': 23, 'right_dentate_nucleus': 24}

Transformed labels:
{'left_caudate': 1, 'right_caudate': 1, 'left_putamen': 2, 'right_putamen': 2, 'left_thalamus': 3, 'right_thalamus': 4, 'left_globus_pallidus': 5, 'right_globus_pallidus': 5, 'internal_capsule': 6, 'left_red_nucleus': 7, 'right_red_nucleus': 8, 'left_substantia_nigra': 9, 'right_substantia_nigra': 9}

Inverse transformed labels:
{'left_caudate': 3, 'right_caudate': 4, 'left_putamen': 5, 'right_putamen': 6, 'left_thalamus': 7, 'right_thalamus': 8, 'left_globus_pallidus': 9, 'right_globus_pallidus': 10, 'internal_capsule': 17, 'left_red_nucleus': 19,

In [ ]:
print(subject["X"])

In [ ]:
for key, value in context.dataset[1].items():
    print(key, value, type(value))
    if isinstance(value, dict):
        for key, value2 in value.items():
            print(key, value2)

In [3]:
def sample_data(loader):
    while True:
        for batch in loader:
            yield batch
           
for i in range(2):
    loader = sample_data(context.dataloader)
    batch = next(loader)
    for key, val in batch.items():
        print(key, val.shape, val.dtype)

X torch.Size([4, 2, 80, 112, 32]) torch.float32
y torch.Size([4, 80, 112, 32]) torch.int32
X torch.Size([4, 2, 80, 112, 32]) torch.float32
y torch.Size([4, 80, 112, 32]) torch.int32


In [ ]:
x = torch.randn(20, 20, 20)
for y in x.split([3, 2, 5, 10]):
    print(y.shape)
